# Transect animation

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob
import matplotlib.colors as col
import matplotlib.path as mpath
from mpl_toolkits.axes_grid1 import make_axes_locatable

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client
figdir = '/g/data/v45/akm157/figures/easterlies/animations/'

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36515 Dashboard: /proxy/8787/status,Cluster Workers: 8 Cores: 48 Memory: 188.57 GiB


In [3]:
# database for control simulation:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
control = '01deg_jra55v13_ryf9091'

# database for perturbations:
easterlies_session = cc.database.create_session('/g/data/v45/akm157/model_data/access-om2/01deg_jra55v13_ryf9091_easterlies_up10/easterlies.db')
# 10% increase in easterly wind speed:
easterlies_up = '01deg_jra55v13_ryf9091_easterlies_up10'
# 10% decrease in easterly wind speed:
easterlies_down = '01deg_jra55v13_ryf9091_easterlies_down10'

# plot anomalies for last 5 years of simulation:
start_time='2160-01-01'
end_time='2164-12-31'

lat_slice  = slice(-80,-50)

## Anomaly transects
I think to see a signal, it may be better to look at monthly averages not daily averages, and calculate the anomaly as a difference from a 10 year monthly mean of the control.

In [52]:
zlevel1 = 0
zlevel2 = 1600

first_year = '2150'
last_year = '2164'

start_time=first_year+'-01-01'
end_time=last_year+'-12-31'

def get_transect_data(lon,lat1,lat2):
    lat_slice  = slice(lat1, lat2)

    # import bathymetry:
    ht_transect = cc.querying.getvar(control,'ht',master_session,n=1)
    ht_transect = ht_transect.sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat_slice)
    ht_transect = ht_transect.load()
    ht_transect[np.isnan(ht_transect)] = 0

    ### calculate 10 year climatology of control:
    temp_con_transect = cc.querying.getvar(control,'temp' , master_session,frequency='1 monthly',start_time=start_time, end_time=end_time) - 273.15
    temp_con_transect = temp_con_transect.sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).sel(st_ocean=slice(zlevel1,zlevel2))
    temp_con_transect = temp_con_transect.groupby('time.month').mean()
    temp_con_transect = temp_con_transect.load()

    temp_up_transect = cc.querying.getvar(easterlies_up,'temp' , easterlies_session,frequency='1 monthly',start_time=start_time, end_time=end_time) - 273.15
    temp_up_transect = temp_up_transect.sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).sel(st_ocean=slice(zlevel1,zlevel2))
    temp_up_transect = temp_up_transect.load()

    ### load data for this period:
    salt_con_transect = cc.querying.getvar(control,'salt' , master_session,frequency='1 monthly',start_time=start_time, end_time=end_time)
    salt_con_transect = salt_con_transect.sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).sel(st_ocean=slice(zlevel1,zlevel2))
    salt_con_transect = salt_con_transect.groupby('time.month').mean()
    salt_con_transect = salt_con_transect.load()

    salt_up_transect = cc.querying.getvar(easterlies_up,'salt' , easterlies_session,frequency='1 monthly',start_time=start_time, end_time=end_time)
    salt_up_transect = salt_up_transect.sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).sel(st_ocean=slice(zlevel1,zlevel2))
    salt_up_transect = salt_up_transect.load()

    ### also get density for plotting isopycnals:
    rho0_con = cc.querying.getvar(control,'pot_rho_0' , master_session,frequency='1 monthly',start_time=start_time, end_time=end_time)
    rho0_con = rho0_con.sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).sel(st_ocean=slice(zlevel1,zlevel2))
    rho0_con = rho0_con.groupby('time.month').mean()
    rho0_con = rho0_con.load()

    rho0_up = cc.querying.getvar(easterlies_up,'pot_rho_0' , easterlies_session,frequency='1 monthly',start_time=start_time, end_time=end_time)
    rho0_up = rho0_up.sel(xt_ocean=lon,method='nearest').sel(yt_ocean=lat_slice).sel(time=slice(start_time,end_time)).sel(st_ocean=slice(zlevel1,zlevel2))
    rho0_up = rho0_up.load()
    
    return ht_transect,temp_con_transect,temp_up_transect,salt_con_transect,salt_up_transect,rho0_con,rho0_up

In [53]:
def plot_transects(region):

    if region == 'AmundsenW':
        lon = -135
        lat1 = -74.7
        lat2 = -73
    elif region == 'AmundsenE':
        lon = -107
        lat1 = -75
        lat2 = -70
    elif region == 'Mertz':
        lon = -220
        lat1 = -66.8
        lat2 = -64.5
    elif region == 'Totten':
        lon = -242
        lat1 = -67.2
        lat2 = -64
    elif region == 'Vincennes':
        lon = -253
        lat1 = -67
        lat2 = -63
    elif region == 'PrydzE':
        lon = 60
        lat1 = -67.5
        lat2 = -64    
    elif region == 'PrydzW':
        lon = 35.5
        lat1 = -69.5
        lat2 = -66 

    ht_transect,temp_con_transect,temp_up_transect,salt_con_transect,salt_up_transect,rho0_con,rho0_up = get_transect_data(lon,lat1,lat2+.1)

    fontsize=16
    scale_max = 0.4
    density_contours = [1027.5,1027.6,1027.72,1027.78,1027.81,1027.83,1027.835,1027.84,1027.845,1027.85]

    for time_step in range(12*15):
    
        fig=plt.figure(figsize=(15,6))

        month_select = temp_up_transect.isel(time=time_step).time.dt.month.values
        year_select = temp_up_transect.isel(time=time_step).time.dt.year.values
        
        print('On year:', year_select, 'month: ',month_select)

        #### Up temp:
        plt.subplot(1,2,1)
        norm = col.Normalize(vmin=-.4,vmax=.4)
        levels=np.arange(-.4,.405,.005)
        ticks = [-.4,-.2,0,.2,.4]
        pcm = plt.contourf(temp_con_transect.yt_ocean,temp_con_transect.st_ocean,
                           temp_up_transect.isel(time=time_step).interpolate_na(dim='st_ocean',method='nearest',fill_value='extrapolate')-\
                           temp_con_transect.sel(month=month_select).interpolate_na(dim='st_ocean',method='nearest',fill_value='extrapolate'),
                           cmap = cmocean.balance, norm = norm,levels=levels,extend='both')
        plt.fill_between(temp_con_transect.yt_ocean,ht_transect,1500,zorder=3,color='darkgrey')
        plt.ylim(1500,0)
        plt.xlim(lat1,lat2)
        plt.xticks([-74.5,-74,-73.5,-73])
        plt.xlabel('Latitude',fontsize=fontsize)
        plt.tick_params(labelsize=fontsize)

        # isopycnals:
        cont=plt.contour(rho0_con.yt_ocean,rho0_con.st_ocean,rho0_up.isel(time=time_step),density_contours,
                          colors='k',zorder=2 ,fontsize=fontsize)
        cont=plt.contour(rho0_con.yt_ocean,rho0_con.st_ocean,rho0_con.sel(month=month_select),density_contours,
                          colors='k',zorder=2 ,fontsize=fontsize,linestyles='--')

        plt.title('Temperature, year '+str(year_select-2150)+', month '+str(month_select),fontsize=fontsize)

        # colorbar:
        ax = plt.gca()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.2)
        cbar = plt.colorbar(pcm, cax=cax,orientation = 'vertical',extend = 'neither',ticks=ticks)
        cbar.set_label(r'UP Temperature anomaly ($^\circ$C)',fontsize=fontsize,labelpad=5)
        cbar.ax.tick_params(labelsize=fontsize)

        #### Up salt:
        plt.subplot(1,2,2)
        norm = col.Normalize(vmin=-.1,vmax=.1)
        levels=np.arange(-.1,.101,.001)
        ticks = [-.1,-.05,0,.05,.1]
        pcm = plt.contourf(salt_con_transect.yt_ocean,salt_con_transect.st_ocean,
                           salt_up_transect.isel(time=time_step).interpolate_na(dim='st_ocean',method='nearest',fill_value='extrapolate')-\
                           salt_con_transect.sel(month=month_select).interpolate_na(dim='st_ocean',method='nearest',fill_value='extrapolate'),
                           cmap = cmocean.balance, norm = norm,levels=levels,extend='both')
        plt.fill_between(salt_con_transect.yt_ocean,ht_transect,1500,zorder=3,color='darkgrey')
        plt.ylim(1500,0)
        plt.xlim(lat1,lat2)
        plt.xticks([-74.5,-74,-73.5,-73])
        plt.xlabel('Latitude',fontsize=fontsize)
        plt.tick_params(labelsize=fontsize)
        plt.title('Salinity',fontsize=fontsize)

        # isopycnals:
        cont=plt.contour(rho0_con.yt_ocean,rho0_con.st_ocean,rho0_up.isel(time=time_step),density_contours,
                          colors='k',zorder=2 ,fontsize=fontsize)
        cont=plt.contour(rho0_con.yt_ocean,rho0_con.st_ocean,rho0_con.sel(month=month_select),density_contours,
                          colors='k',zorder=2 ,fontsize=fontsize,linestyles='--')

        # colorbar:
        ax = plt.gca()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.2)
        cbar = plt.colorbar(pcm, cax=cax,orientation = 'vertical',extend = 'neither',ticks=ticks)
        cbar.set_label(r'UP Salinity anomaly',fontsize=fontsize,labelpad=5)
        cbar.ax.tick_params(labelsize=fontsize)

        plt.figtext(.07, .2, region+',\nlongitude='+str(lon),fontsize=fontsize)

        plt.tight_layout()

        # save frame
        filestr = figdir+'/transect_frames/image%03d.png' % time_step
        plt.savefig(filestr,dpi=150,bbox_inches='tight')
        plt.clf()


In [ ]:
plot_transects('AmundsenW')